In [ ]:
import mne, os, glob
import numpy as np
from joblib import Parallel, delayed
"""Script to create and save ICA fif files for later examination and removal, and filter the data using
a notch filter + a high pass filter"""

# Set base of bids directory here
bids_top_dir = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/'
def load_raw(bids_dir, subjid, session_nr, run_nr, add_dynamic=False):
    "Load raw data"
    task_name = 'OrientationImageryDynamic' if add_dynamic else 'OrientationImagery'
    raw_fn = f'{bids_dir}/sub-{subjid}/ses-{session_nr}/meg/sub-{subjid}_ses-{session_nr}_task-{task_name}_run-{str(run_nr).zfill(2)}_meg.ds'
    raw = mne.io.read_raw_ctf(raw_fn, system_clock='ignore', preload=True)
    return raw, raw_fn


def filter(raw):
    "Filter out mainline frequency data, and apply a high pass filter for frequencies below .5 Hz"
    raw.notch_filter(np.arange(60, 241, 60), phase='zero-double', fir_design='firwin2')
    raw.filter(l_freq=0.5, h_freq=None)
    return raw

def ica(raw, bids_dir, raw_fn, add_dynamic=False):
    "Apply ICA to the raw data and save out the ICA files for later examination, plus save out the filtered raw data"
    ica = mne.preprocessing.ICA(n_components=20, noise_cov=None, random_state=1, 
                                method='fastica', fit_params=None, max_iter='auto', allow_ref_meg=False, verbose=None)
    ica.fit(raw)
    out_dir = f"/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/ICA/"
    os.makedirs(out_dir, exist_ok=True)
    
    base = raw_fn.split('/')[-1][:-7]
    
    ica.save(f"{out_dir}{base}_step1a-ica.fif", overwrite=True)
    raw.save(f"{out_dir}{base}_step1a-raw.fif", overwrite=True)


def run_ica(bids_dir, subjid, session_nr, run_nr, add_dynamic=False):
    "Run through the process of loading data, events, cropping, filtering, and running ICA"
    raw, raw_fn = load_raw(bids_dir, subjid, session_nr, run_nr, add_dynamic=add_dynamic)
    events = mne.events_from_annotations(raw)
    print('events', events)
    print('times', len(raw.times))
    raw.crop(0,raw.times[events[0][-1,0]]+4)
    raw = filter(raw)
    ica(raw, bids_dir, raw_fn, add_dynamic=add_dynamic)


#*****************************#
### Run Script ###
#*****************************#
subjects = [f"S{i:02}" for i in range(1, 21)]
for subject in subjects:
    subjid = subject
    session_nr = '1'

    bids_dir = bids_top_dir.rstrip('/')  
    dsets = []
    #Loop for still runs
    for i in range(1, 5):  
        ds = f'run-{i:02}'
        pattern = f'{bids_dir}/sub-{subjid}/ses-{session_nr}/meg/sub-{subjid}_ses-{session_nr}_task-OrientationImagery_{ds}_meg.ds'
        matches = glob.glob(pattern)
        if matches:
            dsets.append(i)
        else:
            print(f"No files found for pattern: {pattern}")

    print("Datasets to process:", dsets)
    for run_nr in dsets:
        run_ica(bids_dir, subjid, session_nr, run_nr, add_dynamic=False)
    Parallel(n_jobs=12, backend="loky")(
        delayed(run_ica)(bids_dir, subjid, session_nr, run_nr, add_dynamic=False) for run_nr in dsets
    )

    #Loop for Dynamic runs
    dsets = []
    for i in range(1, 5):  
        ds = f'run-{i:02}'
        pattern = f'{bids_dir}/sub-{subjid}/ses-{session_nr}/meg/sub-{subjid}_ses-{session_nr}_task-OrientationImageryDynamic_{ds}_meg.ds'
        matches = glob.glob(pattern)
        if matches:
            dsets.append(i)
        else:
            print(f"No files found for pattern: {pattern}")

    print("Datasets to process:", dsets)

    # Run ICA on available datasets
    Parallel(n_jobs=12, backend="loky")(
        delayed(run_ica)(bids_dir, subjid, session_nr, run_nr, add_dynamic=True) for run_nr in dsets
    )

Datasets to process: [4]
ds directory : /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImageryDynamic_run-04_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
       1.67   64.50    0.00 mm <->    1.67   64.50    0.00 mm (orig :  -37.93   58.24 -252.05 mm) diff =    0.000 mm
      -1.67  -64.50    0.00 mm <->   -1.67  -64.50    0.00 mm (orig :   62.91  -21.87 -260.16 mm) diff =    0.000 mm
      79.79    0.00    0.00 mm <->   79.79    0.00    0.00 mm (orig :   55.17   70.98 -214.17 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.4s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    2.9s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    4.6s


Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.5 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Filter length: 7921 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.3s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    2.6s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    4.1s


Fitting ICA to data using 269 channels (please be patient, this may take a while)
Removing 5 compensators from info because not all compensation channels were picked.
Selecting by number: 20 components
Fitting ICA took 21.6s.
Writing ICA solution to /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/ICA/sub-S04_ses-1_task-OrientationImageryDynamic_run-04_step1a-ica.fif...
Writing /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/ICA/sub-S04_ses-1_task-OrientationImageryDynamic_run-04_step1a-raw.fif
Closing /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/ICA/sub-S04_ses-1_task-OrientationImageryDynamic_run-04_step1a-raw.fif
[done]


In [12]:
import mne, os, glob
import numpy as np
from joblib import Parallel, delayed

"""Script to check MEG sensors and identify missing ones."""

def load_raw(bids_dir, subjid, session_nr, run_nr, add_dynamic=False):
    "Load raw data"
    task_name = 'OrientationImageryDynamic' if add_dynamic else 'OrientationImagery'
    raw_fn = f'{bids_dir}/sub-{subjid}/ses-{session_nr}/meg/sub-{subjid}_ses-{session_nr}_task-{task_name}_run-{str(run_nr).zfill(2)}_meg.ds'
    raw = mne.io.read_raw_ctf(raw_fn, system_clock='ignore', preload=True)
    return raw, raw_fn

# -------------------------
# Main loop
# -------------------------
subjects = [f"S{i:02}" for i in range(1, 21)]
for subject in subjects:
    if subject == 'S03' or subject == 'S05':
        continue  # Skip subject S03 due to known data issues
    subjid = subject
    session_nr = '1'
    run_nr = 1  # example run number
    bids_dir = '/Users/sm6511/Desktop/NIH_Experiment/Bids'

    raw, raw_fn = load_raw(bids_dir, subjid, session_nr, run_nr)

    # --- Count MEG sensors ---
    meg_picks = mne.pick_types(raw.info, meg=True, eeg=False, ref_meg=False)
    ref_meg_picks = mne.pick_types(raw.info, meg=False, ref_meg=True)

    print(f"\n{subjid} run-{run_nr:02} ({'Dynamic' if True else 'Still'})")
    print(f"  Total MEG sensors: {len(meg_picks)}")
    print(f"  Reference MEG sensors: {len(ref_meg_picks)}")

    # --- Identify missing sensors ---
    all_meg_chs = [ch for ch in raw.ch_names if 'M' in ch]  # CTF MEG channels start with M
    expected_meg_chs = [f"M{str(i).zfill(3)}" for i in range(1, 276)]  # expected 275 sensors
    missing_chs = [ch for ch in expected_meg_chs if ch not in all_meg_chs]

    if missing_chs:
        print(f"  ⚠️ Missing sensors ({len(missing_chs)}): {missing_chs}")
    else:
        print("  ✅ No missing MEG sensors detected.")

    # --- Check if any are marked bad ---
    if raw.info["bads"]:
        print(f"  Bad channels: {raw.info['bads']}")


ds directory : /Users/sm6511/Desktop/NIH_Experiment/Bids/sub-S01/ses-1/meg/sub-S01_ses-1_task-OrientationImagery_run-01_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
       3.75   71.65    0.00 mm <->    3.75   71.65   -0.00 mm (orig :  -41.24   60.00 -249.05 mm) diff =    0.000 mm
      -3.75  -71.65    0.00 mm <->   -3.75  -71.65   -0.00 mm (orig :   56.29  -45.24 -250.55 mm) diff =    0.000 mm
      89.55    0.00    0.00 mm <->   89.55   -0.00    0.00 mm (orig :   68.70   70.21 -231.63 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /Users/sm6511/Desktop/NIH_Experiment/Bids/sub-S01/ses-1/meg/sub-S01_ses-1_task-OrientationImagery_run-01_meg.ds/sub-S01_ses-1_task-OrientationImagery_run-01_meg.meg4: 
    System clock channel is